In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option.max_columns = 300

In [ ]:
train_df = pd.read_csv("../input/train_2016.csv", parse_dates=["transactiondate"])

In [ ]:
train_df['transaction_month'] = train_df.transactiondate.dt.month

In [ ]:
train_df.head()

In [ ]:
f,ax = plt.subplots(figsize = (10,8))
plt.scatter(x=range(train_df.shape[0]),y=np.sort(train_df.logerror.values))
plt.show()

In [ ]:
cnt_trans = train_df.transaction_month.value_counts()

In [ ]:
cnt_trans.index

In [ ]:
cnt_trans.values

In [ ]:
plt.figure(figsize=(10,12))
sns.distplot(train_df.logerror.values,bins=50,kde=False)
sns.plt.show()

In [ ]:
plt.figure(figsize=(10,12))
sns.barplot(x=cnt_trans.index,y=cnt_trans.values)
plt.xlabel('MONTH OF TRANSACTION',fontsize=12)
plt.ylabel('NUMBER OF OCCURENCES',fontsize=12)

In [ ]:
prop_df =  pd.read_csv("../input/properties_2016.csv")

In [ ]:
miss_df = prop_df.isnull().sum(axis=0).reset_index()

In [ ]:
miss_df.columns = ['column','miss_count']

In [ ]:
miss_df.head()

In [ ]:
miss1_df = miss_df.sort_values(by='miss_count')

In [ ]:
plt.figure(figsize=(30,40))
sns.set(font_scale=1.5)
sns.barplot(x='miss_count',y='column',data=miss1_df)
sns.plt.show()

In [ ]:
train_df.corr()

In [ ]:
pro_df = prop_df.copy()
tr_df = train_df.copy()

In [ ]:
con_df = pd.merge(tr_df,pro_df,how='left',on='parcelid')

In [ ]:
miss2_df = con_df.isnull().sum(axis=0).reset_index()

In [ ]:
miss2_df.columns = ['col','miss_cnt']

In [ ]:
miss2_df.head()

In [ ]:
miss3_df = miss2_df.sort_values(by='miss_cnt')

In [ ]:
miss3_df.head()

In [ ]:
plt.figure(figsize=(40,50))
sns.set(font_scale=1.5)
sns.barplot(x='miss_cnt',y='col',data=miss3_df)

sns.plt.show()

In [ ]:
corrmat = con_df.corr()

In [ ]:
plt.figure(figsize=(30,40))
sns.heatmap(corrmat,vmax=.8, square=True)
sns.plt.show()

In [ ]:
dtype_df = con_df.dtypes.reset_index()

In [ ]:
dtype_df.columns = ['col','dtyp']

In [ ]:
d = dtype_df.groupby(['dtyp'])['col'].sum().reset_index()

In [ ]:
sns.regplot(x='numberofstories',y='logerror',data=con_df)
sns.plt.show()

In [ ]:
np.mean(con_df['bedroomcnt'])
sns.boxplot(y='bedroomcnt',data=con_df)
sns.plt.show()

In [ ]:
for col in con_df.columns:
    if con_df[col].dtypes=='object':
        print (col)

In [ ]:
sns.regplot(x='bedroomcnt',y='bathroomcnt',data=con_df)
sns.plt.show()

In [ ]:
sns.regplot(x='numberofstories',y='logerror',data=con_df)
sns.plt.show()

In [ ]:
sns.regplot(x='lotsizesquarefeet',y='logerror',data=con_df)
sns.plt.show()

In [ ]:
con1_df = con_df.copy()

In [ ]:
ob_col=[]
for col in con1_df.columns:
    if con1_df[col].dtypes=='object':
        ob_col.append(col)

In [ ]:
ob_col.append('transaction_month')
ob_col.append('transactiondate')

In [ ]:

for col in con1_df:
    if col not in ob_col:
        con1_df[col].fillna(np.mean(con1_df[col]),inplace=True)

In [ ]:
featto_use=[]
for col in con1_df.columns:
    if col not in ob_col:
        featto_use.append(col)

In [ ]:
featto_use.remove('logerror')

In [ ]:
con2_df = con1_df[featto_use]

In [ ]:
X_train,y_train=con2_df,con1_df['logerror']

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
clf = GradientBoostingRegressor()

In [ ]:
clf = clf.fit(X_train,y_train)

In [ ]:
feat_imp_num = list(clf.feature_importances_)

In [ ]:
feat_dict = dict(zip(featto_use,feat_imp_num))

In [ ]:
sorted_feat = reversed(sorted(feat_dict,key=feat_dict.__getitem__))

In [ ]:
feat_name_rank=[]
feat_rank=[]
for k in sorted_feat:
    if feat_dict[k]!=0.0:
        feat_name_rank.append(k)
        feat_rank.append(feat_dict[k])

In [ ]:
rankwise_dict = dict(zip(feat_name_rank,feat_rank))

In [ ]:
feat_df = pd.DataFrame.from_dict(rankwise_dict,orient='index').reset_index()

In [ ]:
feat_df.columns = ['f_name','rank']

In [ ]:
plt.figure(figsize=(20,30))
sns.barplot(x='rank',y='f_name',data=feat_df)
sns.plt.show()

In [ ]:
#please upvote, if you find it useful